see https://github.com/hankcs/udacity-deep-learning/blob/master/6_lstm.py#L627

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0

def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


see https://github.com/ahangchen/GDLnotes/blob/master/src/rnn/embed_bigram_lstm.py

In [6]:
bigram_vocabulary_size =vocabulary_size * vocabulary_size
class BigramBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size_in_chars = len(text)
    self._text_size = self._text_size_in_chars//2
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
   ## """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=self._batch_size, dtype=np.int)
    for b in range(self._batch_size):
      #batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      
        char_idx = self._cursor[b] *2
        ch1 = char2id(self._text[char_idx])
        if self._text_size_in_chars -1 == char_idx:
            ch2 = 0
        else:
            ch2 =char2id(self._text[char_idx +1])
        batch[b] = ch1 * vocabulary_size + ch2
                      
        self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def bi2str(encoding):
    return id2char(encoding // vocabulary_size)+ id2char(encoding % vocabulary_size)
      
def bigrams(encodings):
    return [bi2str(e) for e in encodings]
    
def bibatches2string(batches):
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s,bigrams(b))]
    return s
    
bi_onehot = np.zeros((bigram_vocabulary_size,bigram_vocabulary_size))
np.fill_diagonal(bi_onehot, 1)

def bi_one_hot(encodings):
    return [bi_onehot[e] for e in encodings]

train_batches = BigramBatchGenerator(train_text,8,8)
valid_batches = BigramBatchGenerator(valid_text,1,1)
print(bibatches2string(train_batches.next()))
print(bibatches2string(train_batches.next()))
print(bibatches2string(valid_batches.next()))
print(bibatches2string(valid_batches.next()))


#def characters(probabilities):
#  """Turn a 1-hot encoding or a probability distribution over the possible
#  characters back into its (most likely) character representation."""
#  return [id2char(c) for c in np.argmax(probabilities, 1)]

#def batches2string(batches):
#  """Convert a sequence of batches back into their (most likely) string
#  representation."""
#  s = [''] * batches[0].shape[0]
#  for b in batches:
#    s = [''.join(x) for x in zip(s, characters(b))]
#  return s

#train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
#valid_batches = BatchGenerator(valid_text, 1, 1)

#print(batches2string(train_batches.next()))
#print(batches2string(train_batches.next()))
#print(batches2string(valid_batches.next()))
#print(batches2string(valid_batches.next()))

['ons anarchists adv', 'on from the nation', 'significant than i', 'ain drugs confusio', 'ate of the origina', 't or at least not ', 'he first daily col', 'rdoo ricky ricardo']
['dvocate social rel', 'onal media and fro', ' in jersey and gue', 'ion inability to o', 'nal document fax m', 't parliament s opp', 'ollege newspaper i', 'do this classic in']
[' ana']
['narc']


In [37]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction,  size=vocabulary_size):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def one_hot_voc(prediction, size=vocabulary_size):
    p = np.zeros(shape=[1, size], dtype=np.float)
    p[0, prediction[0]] = 1.0
    return p

def random_distribution(size=vocabulary_size):
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, size])
  return b/np.sum(b, 1)[:,None]

In [16]:
num_nodes = 512
embedding_size = 128
#keep_prob = 0.6
batch_size=32
num_unrollings=10
graph = tf.Graph()

with graph.as_default():
  
  # Parameters:
  embeddings = tf.Variable(
    tf.random_uniform([bigram_vocabulary_size, embedding_size], -1.0, 1.0))
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
    
  #think a lot of time then glance at https://github.com/magicly/udacity-ud730/blob/master/6_lstm.ipynb thanks
  ifcox = tf.concat([ix,fx,cx,ox],1)
  ifcom = tf.concat([im,fm,cm,om],1)
  ifcob = tf.concat([ib,fb,cb,ob],1)
 # print(ifcox.shape)
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_vocabulary_size]))
    # one hot encoding for labels in
  np_one_hot = np.zeros((bigram_vocabulary_size, bigram_vocabulary_size))
  np.fill_diagonal(np_one_hot, 1)
  bigram_one_hot = tf.constant(np.reshape(np_one_hot, -1), dtype=tf.float32,
                                 shape=[bigram_vocabulary_size, bigram_vocabulary_size])
  keep_prob = tf.placeholder(tf.float32)  
  
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
   # print(i.shape)
    #embed = tf.nn.embedding_lookup(embeddings, i)  no need here
    #print(embed.shape)\
    i = tf.nn.dropout(i, keep_prob)
    ifco_input = tf.matmul(i,ifcox) + tf.matmul(o,ifcom) +ifcob
    input_ifco,forget_ifco,update_ifco,output_ifco = tf.split(ifco_input,4,1)
    #print(input_ifco.shape)
    input_gate = tf.sigmoid(input_ifco)
    
    forget_gate = tf.sigmoid(forget_ifco)
    update = update_ifco
    output_gate = tf.sigmoid(output_ifco)
    state = forget_gate * state + input_gate * tf.tanh(update)
    #print((output_gate * tf.tanh(state)).shape)
    output=tf.nn.dropout(output_gate * tf.tanh(state), keep_prob)
    return output, state
  
  # np_embeds = np.zeros((bigram_vocabulary_size, bigram_vocabulary_size))  
 # bigramonehot = tf.constant(np.reshape(np_embeds, -1), dtype=tf.float32, shape=[bigram_vocabulary_size, bigram_vocabulary_size],
                                   #name='bigramonehot')  
  # Input data.
  tf_train_data = tf.placeholder(tf.int32, shape=[num_unrollings + 1, batch_size])
  train_data = list()
  #for i in range(num_unrollings + 1):
  #  train_data.append(
    #  tf.placeholder(tf.int32, shape=[batch_size,vocabulary_size]))# shape no change ,i spend a lot of time to think in here an there!
  for i in tf.split(value=tf_train_data, num_or_size_splits=num_unrollings + 1, axis=0):
        train_data.append(tf.squeeze(i))
  
  train_labels = list()
  train_inputs = train_data[:num_unrollings]
  for l in train_data[1:]:
    train_labels.append(tf.gather(bigram_one_hot, l))  # change to fit bigram
  
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
# python loop used: tensorflow does not support sequential operations yet
  for i in train_inputs:
    # embed input bigrams -> [batch_size, embedding_size]      
 
   #tf.argmax(i,axis=1))#  choose the index of the max value in every row
    output, state = lstm_cell(tf.nn.embedding_lookup(embeddings,i), output, state)
    outputs.append(output)
  #  print(output.shape)
  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(values=train_labels, axis=0), logits=logits)) #no change !!

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)
  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
   # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_input_embedding = tf.nn.embedding_lookup(embeddings,sample_input) #tf.agrmax(sample_input,axis=1))
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input_embedding, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))    
 

In [39]:
num_steps = 4001
summary_frequency = 100
from time import time
start = time()
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
  valid_batches = BigramBatchGenerator(valid_text, 1, 1)  
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
  #  feed_dict = dict()
 #   for i in range(num_unrollings + 1):
#    feed_dict[train_data[i]] = batches[i]
 #   _, l, predictions, lr = session.run(
 #     [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate],
                                            feed_dict={tf_train_data: batches, keep_prob: 0.6})
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = list(batches)[1:]
      labels = np.concatenate([bi_one_hot(l) for l in labels])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          print(bigram_vocabulary_size)
          feed = np.argmax(sample(random_distribution(bigram_vocabulary_size),bigram_vocabulary_size))
          sentence = bi2str(feed)
          reset_sample_state.run()
          for _ in range(49):
            prediction = sample_prediction.eval({sample_input: [feed], keep_prob: 1.0})
            feed = np.argmax(sample(prediction, bigram_vocabulary_size))
            sentence += bi2str(feed)
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1.0})
        valid_logprob = valid_logprob + logprob(predictions, one_hot_voc(b[1], bigram_vocabulary_size))
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))
      end = time()
      total = end -start
      print(total)

Initialized
Average loss at step 0: 6.786360 learning rate: 10.000000
Minibatch perplexity: 885.68
729
gfe kjthe zbcypke aiale d mxnwe bqolwce kjdmjve qle nycye cnrje oskmghkbe tmtyvbogpfe soace tmcqetcs
729
zqqze hxoxpftae xef e sut  swye yjjse ciyje mnqgyuehfce awdie kye qfkje qgsde cze kpshe txnnvse yce 
729
pienvne zqsse pnpte pukse ovpajxace dae kzyhgvfnnpdbqfiue dlwpe bdofe  rwme pevte rneue lre qte khcq
729
fle jymde qde lie wilh rwoe nde mle mpuce vxjle gthpgzber bwfme uawoiue  lqpe zde lttcsue irbsqopze 
729
fpxchpe ykdee jpzmcmhi kjpe djcne misse gfe qkgovkfovplde gpide lce cnwte wkensyany vewexbfyxee asyl
Validation set perplexity: 8064.18
2.1098215579986572
Average loss at step 100: 6.389109 learning rate: 10.000000
Minibatch perplexity: 171.89
Validation set perplexity: 155.14
5.243572473526001
Average loss at step 200: 4.846695 learning rate: 10.000000
Minibatch perplexity: 97.28
Validation set perplexity: 98.15
8.339318752288818
Average loss at step 300: 4.505004 learnin

Average loss at step 3900: 3.434409 learning rate: 10.000000
Minibatch perplexity: 32.54
Validation set perplexity: 22.82
123.4099748134613
Average loss at step 4000: 3.517139 learning rate: 10.000000
Minibatch perplexity: 36.25
729
kritions by bbarger flortury the a cany de singeshin addered by american state modspladels are mah m
729
zcpolatical king chaent noweder hnasicanglosue f chass to valle neyer in haurr zeg by labion h ibrea
729
fctism on thatland uned parate stripft hels one five zero zero george stato lack is in hember painte
729
lstury language chark spainsto air quise is mariar vgarcined a an numbernoted fiskial work instant t
729
nhy aeayile have scils stated in stennaven b one eight nine three one six jabil in massan disording 
Validation set perplexity: 22.31
126.71667528152466
